In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import parameters_MF_MB as PRM
import simulations_MF_MB as SIM
import analyzes_MF_MB as ALY
import figures_indiv as GRAPH_i
import figures_pop as GRAPH_p
import figure_qvalue_map as GRAHP_Qmap
# import importlib
# importlib.reload(SIM)

# Reloading modules automatically :
%load_ext autoreload
%autoreload 2

In [2]:
print(PRM.params) # initial set of PRM for reference

{'n_individuals': 100, 'n_trials': 50, 'n_individuals_sims': {'learning': 100, 'generalization': 100, 'fast_test': 10}, 'n_trials_sims': {'learning': 50, 'generalization': 6, 'fast_test': 10}, 'alpha': 0.6, 'beta': 15.0, 'gamma': 0.9, 'alpha_vals': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]), 'beta_vals': {'learning': 15.0, 'generalization': 10.0, 'fast_test': 15.0}, 's_start': 35, 's_rw': 22, 'starting_points': {'learning': 35, 'generalization': [21, 34, 4], 'fast_test': 35}, 'reward_states': [22, 4], 'trial_change': 25, 'RSS': 90, 'RR': 20, 'n_seq_d': 6, 'RSS_d': 15, 'epsilon_d': 0.5, 'epsilon_p': 0.001, 'perc': 0.7, 'window': 5, 'n_types_replays': 4, 'replay_types': {0: 'No replay', 1: 'Backward replay', 2: 'Random replay', 3: 'Most diverse replay', 4: 'Prioritized sweeping'}, 'replay_refs': [0, 1, 2, 3, 4], 'nA': 8, 'nS': 36, 'state_coords': [(0.00320002, 0.0059351), (0.310727, 0.0241474), (0.593997, 0.152759), (0.724483, -0.118826), (0.956215, 0.0761724), (-0.05

# Setting environment and parameters

In [22]:
det = True

params = PRM.params.copy()
params = PRM.set_environment(params, deterministic=det)

# Labels for saving data
if det:
    env = '_D'
else:
    env = '_S'

In [ ]:
params['RR'] = 3

lab = 'RR3'

# Learning

Store the Q matrices of the first learning phase (before change of reward location), to be reused in the following generalization.

## Raw data

In [ ]:
params = PRM.set_simulation(params, 'learning')

Data, Model = SIM.simulate_population(params)

SIM.save_data(Data, 'Data_l'+env)
SIM.save_data(Model, 'Model_l'+env, df=False)

## Learning curves - Performance of the population for each trial

In [13]:
Data = SIM.recover_data('Data_l'+env)

LC_D = ALY.compute_performance_in_time(Data, params)

SIM.save_data(LC, 'LC_l'+env)

## Slitting in periods, before and after change of reward state

In [14]:
Data = SIM.recover_data('Data_l'+env)

Data0, Data1, params0, params1 = ALY.split_before_after_change(Data)

SIM.save_data(Data_l0_D, 'Data_l0'+env)
SIM.save_data(Data_l1_D, 'Data_l1'+env)

/media/esther/Data/Travail/Stages/Navigation/Codes/analyzes_MF_MB.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data1['Trial'] -= params['trial_change'] # reset trial numbers to start from 0


## Performance of individuals across trials

In [34]:
Data = SIM.recover_data('Data_l'+env)
Data0 = SIM.recover_data('Data_l0'+env)
Data1 = SIM.recover_data('Data_l1'+env)

PRF = ALY.compute_performance_across_population(Data, params=params)
PRF0 = ALY.compute_performance_across_population(Data0, params=params0)
PRF1 = ALY.compute_performance_across_population(Data1, params=params1)

SIM.save_data(PF, 'PRF_l'+env)
SIM.save_data(PRF0, 'PRF_l0'+env)
SIM.save_data(PRF1, 'PRF_l1'+env)

## Convergence times

It only makes sense to compute convergence on separate phases of the simulations, on which the reward state is kept constant. Therefore, only two files are produced.

In [15]:
Data0 = SIM.recover_data('Data_l0'+env)
Data1 = SIM.recover_data('Data_l1'+env)

CV0, NonCV0 = ALY.compute_convergence(Data0, params=params0)
CV1, NonCV1 = ALY.compute_convergence(Data1, params=params1)

SIM.save_data(CV_det0, 'CV_l0'+env)
SIM.save_data(CV_det1, 'CV_l1'+env)
SIM.save_data(Non_cv_det0, 'NonCV_l0'+env, df=False)
SIM.save_data(Non_cv_det1, 'NonCV_l1'+env, df=False)

# Generalization

Use the Q matrices obtained at the end of the first learning epoch.

## Raw data

In [ ]:
Model = SIM.recover_data('Model_l'+env, df=False)

for g in range(len(params['starting_points']['generalization'])):
    params = PRM.set_simulation(params, sim='generalization', gen_test=g)

    Data = SIM.simulate_generalization(params, Model)
    
    SIM.save_data(Data, 'Data_g{}'+env.format(g))

## Learning curves - Performance of the population for each trial

In [13]:
for g in range(len(params['starting_points']['generalization'])):
    params = PRM.set_simulation(params, sim='generalization', gen_test=g)
    Data = SIM.recover_data('Data_g{}'+env.format(g))

    LC = ALY.compute_performance_in_time(Data, params)

    SIM.save_data(LC, 'LC_g{}'.format(g)+env)

## Performance of individuals across trials

In [34]:
for g in range(len(params['starting_points']['generalization'])):
    params = PRM.set_simulation(params, sim='generalization', gen_test=g)
    Data = SIM.recover_data('Data_g{}'+env.format(g))

    PRF = ALY.compute_performance_across_population(Data, params=params)

    SIM.save_data(PF, 'PRF_g{}'.format(g)+env)

## Convergence times

It only makes sense to compute convergence on separate phases of the simulations, on which the reward state is kept constant. Therefore, only two files are produced.

In [15]:
Data0 = SIM.recover_data('Data_l0'+env)
Data1 = SIM.recover_data('Data_l1'+env)

CV0, NonCV0 = ALY.compute_convergence(Data0, params=params0)
CV1, NonCV1 = ALY.compute_convergence(Data1, params=params1)

SIM.save_data(CV_det0, 'CV_l0'+env)
SIM.save_data(CV_det1, 'CV_l1'+env)
SIM.save_data(Non_cv_det0, 'NonCV_l0'+env, df=False)
SIM.save_data(Non_cv_det1, 'NonCV_l1'+env, df=False)

# Try with lower number of replay for each sequence

In [16]:
params = PRM.params.copy() # reset parameters
params = PRM.set_environment(params, deterministic=True)
params = PRM.set_simulation(params, 'learning')
params['RR'] = 3
Data_det_RRlow, Model_det_RRlow = SIM.simulate_population(params)
SIM.save_data(Data_det_RRlow, 'Data_det_RRlow' )
SIM.save_data(Model_det_RRlow, 'Model_det_RRlow', df=False)

Perf_det_time_RRlow = ALY.compute_performance_in_time(Data_det_RRlow, params)
SIM.save_data(Perf_det_time_RRlow, 'Perf_det_time_RRlow' )
Perf_det_indiv_RRlow = ALY.compute_performance_across_population(Data_det_RRlow, params=params)
SIM.save_data(Perf_det_indiv_RRlow, 'Perf_det_indiv_RRlow')

Transition matrix set to deterministic in params.
LEARNING...
Replay | Individual
--------------------
0 | 0
0 | 1
0 | 2
0 | 3
0 | 4
0 | 5
0 | 6
0 | 7
0 | 8
0 | 9
0 | 10
0 | 11
0 | 12
0 | 13
0 | 14
0 | 15
0 | 16
0 | 17
0 | 18
0 | 19
0 | 20
0 | 21
0 | 22
0 | 23
0 | 24
0 | 25
0 | 26
0 | 27
0 | 28
0 | 29
0 | 30
0 | 31
0 | 32
0 | 33
0 | 34
0 | 35
0 | 36
0 | 37
0 | 38
0 | 39
0 | 40
0 | 41
0 | 42
0 | 43
0 | 44
0 | 45
0 | 46
0 | 47
0 | 48
0 | 49
0 | 50
0 | 51
0 | 52
0 | 53
0 | 54
0 | 55
0 | 56
0 | 57
0 | 58
0 | 59
0 | 60
0 | 61
0 | 62
0 | 63
0 | 64
0 | 65
0 | 66
0 | 67
0 | 68
0 | 69
0 | 70
0 | 71
0 | 72
0 | 73
0 | 74
0 | 75
0 | 76
0 | 77
0 | 78
0 | 79
0 | 80
0 | 81
0 | 82
0 | 83
0 | 84
0 | 85
0 | 86
0 | 87
0 | 88
0 | 89
0 | 90
0 | 91
0 | 92
0 | 93
0 | 94
0 | 95
0 | 96
0 | 97
0 | 98
0 | 99
--------------------
1 | 0
1 | 1
1 | 2
1 | 3
1 | 4
1 | 5
1 | 6
1 | 7
1 | 8
1 | 9
1 | 10
1 | 11
1 | 12
1 | 13
1 | 14
1 | 15
1 | 16
1 | 17
1 | 18
1 | 19
1 | 20
1 | 21
1 | 22
1 | 23
1 | 24
1 | 25
1 | 26
1 | 27


In [17]:
params = PRM.params.copy()
params['RR'] = 3
params = PRM.set_simulation(params, 'generalization')
Data_det_RRlow_g = SIM.simulate_generalization(params, Model_det_RRlow)
SIM.save_data(Data_det_RRlow_g, 'Data_det_RRlow_g' )

Perf_det_time_RRlow_g = ALY.compute_performance_in_time(Data_det_RRlow_g, params)
SIM.save_data(Perf_det_time_RRlow_g, 'Perf_det_time_RRlow_g' )
Perf_det_indiv_RRlow_g = ALY.compute_performance_across_population(Data_det_RRlow_g, params=params)
SIM.save_data(Perf_det_indiv_RRlow_g, 'Perf_det_indiv_RRlow_g')

GENERALIZATION...
Replay | Individual
--------------------
0 | 0
0 | 1
0 | 2
0 | 3
0 | 4
0 | 5
0 | 6
0 | 7
0 | 8
0 | 9
0 | 10
0 | 11
0 | 12
0 | 13
0 | 14
0 | 15
0 | 16
0 | 17
0 | 18
0 | 19
0 | 20
0 | 21
0 | 22
0 | 23
0 | 24
0 | 25
0 | 26
0 | 27
0 | 28
0 | 29
0 | 30
0 | 31
0 | 32
0 | 33
0 | 34
0 | 35
0 | 36
0 | 37
0 | 38
0 | 39
0 | 40
0 | 41
0 | 42
0 | 43
0 | 44
0 | 45
0 | 46
0 | 47
0 | 48
0 | 49
0 | 50
0 | 51
0 | 52
0 | 53
0 | 54
0 | 55
0 | 56
0 | 57
0 | 58
0 | 59
0 | 60
0 | 61
0 | 62
0 | 63
0 | 64
0 | 65
0 | 66
0 | 67
0 | 68
0 | 69
0 | 70
0 | 71
0 | 72
0 | 73
0 | 74
0 | 75
0 | 76
0 | 77
0 | 78
0 | 79
0 | 80
0 | 81
0 | 82
0 | 83
0 | 84
0 | 85
0 | 86
0 | 87
0 | 88
0 | 89
0 | 90
0 | 91
0 | 92
0 | 93
0 | 94
0 | 95
0 | 96
0 | 97
0 | 98
0 | 99
--------------------
1 | 0
1 | 1
1 | 2
1 | 3
1 | 4
1 | 5
1 | 6
1 | 7
1 | 8
1 | 9
1 | 10
1 | 11
1 | 12
1 | 13
1 | 14
1 | 15
1 | 16
1 | 17
1 | 18
1 | 19
1 | 20
1 | 21
1 | 22
1 | 23
1 | 24
1 | 25
1 | 26
1 | 27
1 | 28
1 | 29
1 | 30
1 | 31
1 | 32
1 | 33
1 

# Testing alpha parameter

## Computing full raw data

In [5]:
alpha_vals = PRM.params['alpha_vals']
print('Alpha values:', alpha_vals)

Alpha values: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [6]:
params = PRM.params.copy()
params = PRM.set_environment(deterministic=True, params=params)
for alpha in alpha_vals:
    print('alpha = {}'.format(alpha))
    params['alpha'] = alpha # set parameter value
    Data_det, _ = SIM.simulate_population(params) # pass the modified dictionary in argument
    SIM.save_data(Data_det, 'Data_det_a{}'.format(alpha))

Transition matrix set to deterministic in params.
alpha = 0.5
LEARNING...
Replay | Individual
--------------------
0 | 0
0 | 1
0 | 2
0 | 3
0 | 4
0 | 5
0 | 6
0 | 7
0 | 8
0 | 9
0 | 10
0 | 11
0 | 12
0 | 13
0 | 14
0 | 15
0 | 16
0 | 17
0 | 18
0 | 19
0 | 20
0 | 21
0 | 22
0 | 23
0 | 24
0 | 25
0 | 26
0 | 27
0 | 28
0 | 29
0 | 30
0 | 31
0 | 32
0 | 33
0 | 34
0 | 35
0 | 36
0 | 37
0 | 38
0 | 39
0 | 40
0 | 41
0 | 42
0 | 43
0 | 44
0 | 45
0 | 46
0 | 47
0 | 48
0 | 49
0 | 50
0 | 51
0 | 52
0 | 53
0 | 54
0 | 55
0 | 56
0 | 57
0 | 58
0 | 59
0 | 60
0 | 61
0 | 62
0 | 63
0 | 64
0 | 65
0 | 66
0 | 67
0 | 68
0 | 69
0 | 70
0 | 71
0 | 72
0 | 73
0 | 74
0 | 75
0 | 76
0 | 77
0 | 78
0 | 79
0 | 80
0 | 81
0 | 82
0 | 83
0 | 84
0 | 85
0 | 86
0 | 87
0 | 88
0 | 89
0 | 90
0 | 91
0 | 92
0 | 93
0 | 94
0 | 95
0 | 96
0 | 97
0 | 98
0 | 99
--------------------
1 | 0
1 | 1
1 | 2
1 | 3
1 | 4
1 | 5
1 | 6
1 | 7
1 | 8
1 | 9
1 | 10
1 | 11
1 | 12
1 | 13
1 | 14
1 | 15
1 | 16
1 | 17
1 | 18
1 | 19
1 | 20
1 | 21
1 | 22
1 | 23
1 | 24
1 | 25
1 

In [7]:
params = PRM.params.copy()
params = PRM.set_environment(deterministic=False, params=params)
for alpha in alpha_vals:
    print('alpha = {}'.format(alpha))
    params['alpha'] = alpha # set parameter value
    Data_sto, _= SIM.simulate_population(params) # pass the modified dictionary in argument
    SIM.save_data(Data_sto, 'Data_sto_a{}'.format(alpha))

Transition matrix set to stochastic in params.
alpha = 0.5
LEARNING...
Replay | Individual
--------------------
0 | 0
0 | 1
0 | 2
0 | 3
0 | 4
0 | 5
0 | 6
0 | 7
0 | 8
0 | 9
0 | 10
0 | 11
0 | 12
0 | 13
0 | 14
0 | 15
0 | 16
0 | 17
0 | 18
0 | 19
0 | 20
0 | 21
0 | 22
0 | 23
0 | 24
0 | 25
0 | 26
0 | 27
0 | 28
0 | 29
0 | 30
0 | 31
0 | 32
0 | 33
0 | 34
0 | 35
0 | 36
0 | 37
0 | 38
0 | 39
0 | 40
0 | 41
0 | 42
0 | 43
0 | 44
0 | 45
0 | 46
0 | 47
0 | 48
0 | 49
0 | 50
0 | 51
0 | 52
0 | 53
0 | 54
0 | 55
0 | 56
0 | 57
0 | 58
0 | 59
0 | 60
0 | 61
0 | 62
0 | 63
0 | 64
0 | 65
0 | 66
0 | 67
0 | 68
0 | 69
0 | 70
0 | 71
0 | 72
0 | 73
0 | 74
0 | 75
0 | 76
0 | 77
0 | 78
0 | 79
0 | 80
0 | 81
0 | 82
0 | 83
0 | 84
0 | 85
0 | 86
0 | 87
0 | 88
0 | 89
0 | 90
0 | 91
0 | 92
0 | 93
0 | 94
0 | 95
0 | 96
0 | 97
0 | 98
0 | 99
--------------------
1 | 0
1 | 1
1 | 2
1 | 3
1 | 4
1 | 5
1 | 6
1 | 7
1 | 8
1 | 9
1 | 10
1 | 11
1 | 12
1 | 13
1 | 14
1 | 15
1 | 16
1 | 17
1 | 18
1 | 19
1 | 20
1 | 21
1 | 22
1 | 23
1 | 24
1 | 25
1 | 2

## Optimizing parameter

In [4]:
params = PRM.params.copy()
Data_det_a_all = ALY.compare_alpha_replays(deterministic=True, params=params)

Computing...
alpha 0.0


/home/esther/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alpha 0.1
alpha 0.2
alpha 0.3
alpha 0.4
alpha 0.5
alpha 0.6
alpha 0.7
alpha 0.8
alpha 0.9
alpha 1.0
Done


In [5]:
SIM.save_data(Data_det_a_all, 'Data_det_a_all')
Data_det_a_all

,Replay type,alpha,Mean,STD,Q1,Q2,Q3
0,0,0.0,121.4524,14.111311,111.305,122.55,129.975
1,0,0.1,81.3738,13.524854,72.600,81.54,88.620
2,0,0.2,62.9246,16.205504,51.790,62.40,73.325
3,0,0.3,54.8808,18.699882,40.575,51.23,66.170
4,0,0.4,45.2118,14.319199,35.140,42.74,51.680
5,0,0.5,39.7698,13.543082,30.165,36.92,44.935
6,0,0.6,37.7174,12.866745,29.160,36.22,44.110
7,0,0.7,35.5510,14.196534,25.115,32.01,40.210
8,0,0.8,35.2956,13.376539,26.495,31.78,41.395
9,0,0.9,33.9510,11.569900,24.935,31.50,41.455


In [6]:
Data_sto_a_all = ALY.compare_alpha_replays(deterministic=False, params=params)

Computing...
alpha 0.0
alpha 0.1
alpha 0.2
alpha 0.3
alpha 0.4
alpha 0.5
alpha 0.6
alpha 0.7
alpha 0.8
alpha 0.9
alpha 1.0
Done


In [7]:
SIM.save_data(Data_sto_a_all, 'Data_sto_a_all')

In [8]:
Data_a_summary = ALY.optimize_alpha(params=params)

Computing...
alpha 0.0
alpha 0.1
alpha 0.2
alpha 0.3
alpha 0.4
alpha 0.5
alpha 0.6
alpha 0.7
alpha 0.8
alpha 0.9
alpha 1.0
Done


In [9]:
SIM.save_data(Data_a_summary, 'Data_a_summary')
Data_a_summary

,alpha,Mean det,STD det,Q1 det,Q2 det,Q3 det,Mean sto,STD sto,Q1 sto,Q2 sto,Q3 sto,Mean tot,STD tot,Q1 tot,Q2 tot,Q3 tot
0,0.0,101.65872,41.781387,99.900,117.42,127.660,100.62028,32.723866,93.725,110.36,122.500,101.13950,37.530487,97.660,113.08,126.180
1,0.1,74.85556,83.717298,24.710,48.15,85.190,97.91472,106.326384,45.755,68.19,95.345,86.38514,96.383993,33.610,61.58,89.760
2,0.2,58.53464,63.217494,21.375,38.28,67.035,74.81784,65.759085,36.615,58.64,82.380,66.67624,65.012615,28.290,48.14,75.785
3,0.3,48.92044,44.615256,20.985,35.19,58.315,64.60564,51.384001,35.500,52.97,72.640,56.76304,48.753715,27.515,45.09,66.710
4,0.4,45.25092,46.616905,20.155,31.94,50.710,63.10092,50.046078,35.075,50.83,72.385,54.17592,49.178537,26.455,41.52,63.850
5,0.5,42.06952,36.994200,19.740,30.10,49.945,57.41268,38.706195,33.420,46.54,66.560,49.74110,38.629307,25.220,38.72,59.160
6,0.6,38.94020,33.652787,18.960,27.82,45.740,58.68376,44.903703,33.435,43.75,65.860,48.81198,40.888609,24.515,37.16,57.420
7,0.7,39.37844,37.724066,18.505,26.53,45.060,56.07680,37.152313,33.920,45.35,65.850,47.72762,38.358944,23.500,36.83,57.745
8,0.8,36.29616,30.109414,18.310,26.32,41.485,54.34440,30.592221,33.575,47.62,66.185,45.32028,31.664888,23.625,36.50,57.580
9,0.9,35.53288,27.479932,18.480,26.23,41.625,58.03648,38.087302,33.760,49.04,68.945,46.78468,35.064193,23.385,36.60,57.465


## Faster computations (direclty storing summary data)

In [44]:
params = PRM.params.copy()
params = PRM.set_environment(deterministic=True, params=params)
Data_det_alpha = SIM.test_alpha(alpha_vals=[0.5,0.8], params=params)

Transition matrix set to deterministic in params_learn.
TEST ALPHA...
Replay | alpha | Individual
--------------------
0 | 0.5 | 0
0 | 0.5 | 1
0 | 0.5 | 2
0 | 0.5 | 3
0 | 0.5 | 4
0 | 0.5 | 5
0 | 0.5 | 6
0 | 0.5 | 7
0 | 0.5 | 8
0 | 0.5 | 9
0 | 0.8 | 0
0 | 0.8 | 1
0 | 0.8 | 2
0 | 0.8 | 3
0 | 0.8 | 4
0 | 0.8 | 5
0 | 0.8 | 6
0 | 0.8 | 7
0 | 0.8 | 8
0 | 0.8 | 9
--------------------
1 | 0.5 | 0
1 | 0.5 | 1
1 | 0.5 | 2
1 | 0.5 | 3
1 | 0.5 | 4
1 | 0.5 | 5
1 | 0.5 | 6
1 | 0.5 | 7
1 | 0.5 | 8
1 | 0.5 | 9
1 | 0.8 | 0
1 | 0.8 | 1
1 | 0.8 | 2
1 | 0.8 | 3
1 | 0.8 | 4
1 | 0.8 | 5
1 | 0.8 | 6
1 | 0.8 | 7
1 | 0.8 | 8
1 | 0.8 | 9
--------------------
2 | 0.5 | 0
2 | 0.5 | 1
2 | 0.5 | 2
2 | 0.5 | 3
2 | 0.5 | 4
2 | 0.5 | 5
2 | 0.5 | 6
2 | 0.5 | 7
2 | 0.5 | 8
2 | 0.5 | 9
2 | 0.8 | 0
2 | 0.8 | 1
2 | 0.8 | 2
2 | 0.8 | 3
2 | 0.8 | 4
2 | 0.8 | 5
2 | 0.8 | 6
2 | 0.8 | 7
2 | 0.8 | 8
2 | 0.8 | 9
--------------------
3 | 0.5 | 0
3 | 0.5 | 1
3 | 0.5 | 2
3 | 0.5 | 3
3 | 0.5 | 4
3 | 0.5 | 5
3 | 0.5 | 6
3 | 0.5 | 7
3 

In [ ]:
SIM.save_data(Data_det_alpha, 'Data_det_a_all' )

In [46]:
Data_det_alpha

,Replay type,alpha,Individual,Performance
0,0,0.5,0,19.76
1,0,0.5,1,53.36
2,0,0.5,2,33.22
3,0,0.5,3,34.12
4,0,0.5,4,28.76
...,...,...,...,...
95,4,0.8,5,54.96
96,4,0.8,6,12.64
97,4,0.8,7,56.52
98,4,0.8,8,26.62
